In [1]:
import pandas as pd

# For this part, we read in authors data and update bibtex key.

In [30]:
mergeable = pd.read_csv("merged_pubs.csv")
authors = pd.read_csv("authors.csv")
authors_pubs = pd.read_csv("authors_publications.csv")

In [31]:
need_fix = ~mergeable["bibtexkey"].str.contains('[A-Za-z]')
to_fix = mergeable[need_fix]
rest = mergeable[~need_fix]

In [32]:
to_fix_with_join = to_fix.merge(authors_pubs, how = 'inner', left_on = "id", right_on = 'publication_id')

In [33]:
to_fix_with_authors = to_fix_with_join.merge(authors, how = "inner", left_on = "author_id", right_on = "id")

In [34]:
to_fix_grouped = to_fix_with_authors.groupby("id_x").first()
to_fix_grouped

,designation,bibtexkey,year,entry_type_id,address,annote,book_title,chapter,crossref,edition,...,first,east_asian_order,email,institution_y,biography,modified,orcid_id,deceased,ojs_author_id,org_script_name
id_x,,,,,,,,,,,,,,,,,,,,,
19909,MAOG 04,1928-1929-19909,1928-1929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
62371,CDLB 2021,2021-62371,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,2022-01-26 09:15:47,NaN,0,NaN,NaN
64491,HMA 9,-64491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,0000-00-00 00:00:00,NaN,0,NaN,NaN
75863,AOAT 046,-75863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,0000-00-00 00:00:00,NaN,0,NaN,NaN
78553,ARET 10,-78553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,0000-00-00 00:00:00,NaN,0,NaN,NaN
86457,WO 06,-86457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,0000-00-00 00:00:00,NaN,0,NaN,NaN
86749,UET 4,-86749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,0000-00-00 00:00:00,NaN,0,NaN,NaN
99777,NABU 2002,2002-99777,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
108418,CT 53,1978-108418,1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


# We ended up filtering out 2 publications that have no author attached as well

In [35]:
not_none = (to_fix_grouped["last"].fillna("").str.len() > 0) & (to_fix_grouped["year"].fillna("").str.len() > 0)

In [36]:
to_fix_grouped.loc[not_none, "bibtexkey"] = to_fix_grouped.loc[not_none, "last"] + to_fix_grouped.loc[not_none, "year"]

In [37]:
fixed_merged = pd.concat([to_fix_grouped, rest], axis = 'index').sort_index()

In [38]:
mergeable = mergeable.set_index("id")

In [39]:
mergeable.loc[to_fix_grouped["bibtexkey"].index, "bibtexkey"] = to_fix_grouped["bibtexkey"]

In [41]:
mergeable.to_csv("merged_pubs.csv")

# We also deal with duplicate bibtexkeys here.

In [3]:
mergeable = pd.read_csv("merged_pubs.csv")

In [57]:
for bibtexkey in mergeable["bibtexkey"].unique():
    occurences = sum(mergeable["bibtexkey"] == bibtexkey)
    if occurences <= 1:
        continue
    idx = 0
    for id, row in mergeable[mergeable["bibtexkey"] == bibtexkey].iterrows():
        mergeable.loc[id, "bibtexkey"] += chr(ord('a') + idx)
        idx += 1

In [61]:
mergeable.to_csv("merged_pubs.csv")